In [1]:
from utils.engine import RFSampler
from model.UNet import UNet
import torch
from utils.tools import save_sample_image, save_image, generator_parse_option, generate_batches
from utils.filter import filter_images_by_cosine_similarity, check_image_counts, trim_images_to_x_per_class
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import os
import sys
from utils.callbacks import set_seed
from utils.RectifiedFlow import RectifiedFlow

In [2]:
set_seed(42)  # 设置随机种子为42

In [3]:
# 模拟命令行输入
sys.argv = [
    'generate.py', 
    '-cp', 'checkpoint/cwru_rf/cwru_rf_br1_5_500epoch.pth', 
    '--device', 'cuda', 
    '--sampler', 'rf', 
    '--model', 'unet',
    '-bs', '64', 
    '--interval', '5', 
    '--eta', '0.0', 
    '--steps', '10', 
    '--method', 'quadratic', 
    '--nrow', '25', 
    '-sp', 'data/cwru_rf_result/cwru_sampler_br1_5_500epoch', 
    '-if', 'data/cwru/test',
    '-mp', 'checkpoint/cwru_rf/match_list.pth',
    '--cosine_threshold', '0.85',
    '--num_batches', '10',
    '--cfg_scale', '1.0',
    '--target_class_count', '2000',
    '--num_classes', '10'
]

In [4]:
@torch.no_grad()
def generate(args):  
    device = torch.device(args.device)

    # Load checkpoint
    cp = torch.load(args.checkpoint_path)
    # Load trained model
    if args.model == 'unet':
       model = UNet(**cp["config"]["Model"])
    model.load_state_dict(cp["model"])
    model.to(device)
    model.eval()

    if args.sampler == "rf":
       sampler = RFSampler(model).to(device)
       extra_param = dict(steps=args.steps, cfg_scale = args.cfg_scale)
       match_list = []
    else:
        raise ValueError(f"Unknown sampler: {args.sampler}")

    # Initialize a dictionary to track the number of images generated for each class
    generated_class_counts = {i: 0 for i in range(args.num_classes)}
    
    # Calculate how many batches you want to generate
    num_batches = args.num_batches  # Specify how many batches you want to generate
    labels = generate_batches(num_batches, cp["config"]["Dataset"]["batch_size"], args.num_classes)
    for batch_idx in range(num_batches):
        print(f"Generating batch {batch_idx + 1}/{num_batches}...")

        # Generate new Gaussian noise for each batch to increase diversity
        z_t = torch.randn((args.batch_size, cp["config"]["Model"]["in_channels"],
                           *cp["config"]["Dataset"]["image_size"]), device=device)

        # Generate images using the sampler
        x, label = sampler(z_t, batch_idx, labels, only_return_x_0=args.result_only, interval=args.interval, **extra_param)
        print(f"Generated {x.shape[0]} images of size {x.shape[1:]}")

        # Save generated images based on the result flag
        if args.result_only:
            # Save images for each batch, passing the batch_idx
            save_image(x, labels=label, nrow=args.nrow, show=args.show, 
                       path=args.image_save_path, to_grayscale=args.to_grayscale, batch_idx=batch_idx)
        else:
            # Save intermediate images for each batch
            save_sample_image(x, labels=label, show=args.show, 
                              path=args.image_save_path, to_grayscale=args.to_grayscale, batch_idx=batch_idx, 
                              save_as_gif=args.gif, gif_duration=args.gif_speed, max_steps=args.max_steps)

        if args.result_only == True:
            filter_images_by_cosine_similarity(args.original_image_folder, args.image_save_path, batch_id=batch_idx, num_classes=args.num_classes, threshold=args.cosine_threshold)
        
        # 在调用 check_image_counts 前打印一些信息
        print(f"检查图像数量：{args.image_save_path}")
        if check_image_counts(args.image_save_path, args.target_class_count, args.num_classes):
            print(f"所有类别图像数均超过 {args.target_class_count}，终止生成。")
            break  # 退出外层循环
        if args.sampler == "rf":
            match = (x,z_t,label)
            match_list.append(match)

    if args.result_only == True:
       # 在生成后检查并删减多余图像
       trim_images_to_x_per_class(args.image_save_path, args.target_class_count, args.num_classes)
    if args.sampler == "rf":
       torch.save(match_list,args.match_save_path)
        


In [5]:
args = generator_parse_option()
generate(args)

Generating batch 1/10...


100%|██████████| 10/10 [00:03<00:00,  2.55it/s, step=1, sample=1]


Generated 64 images of size torch.Size([3, 32, 32])
文件夹不存在，跳过类 0：data/cwru/test\0 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\0
文件夹不存在，跳过类 1：data/cwru/test\1 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\1
文件夹不存在，跳过类 2：data/cwru/test\2 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\2
文件夹不存在，跳过类 3：data/cwru/test\3 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\3
文件夹不存在，跳过类 4：data/cwru/test\4 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\4
文件夹不存在，跳过类 5：data/cwru/test\5 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\5
文件夹不存在，跳过类 6：data/cwru/test\6 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\6
文件夹不存在，跳过类 7：data/cwru/test\7 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\7
文件夹不存在，跳过类 8：data/cwru/test\8 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\8
文件夹不存在，跳过类 9：data/cwru/test\9 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\9
共删除 0 张生成图片。
检查图像数量：data/cwru_rf_result/cwru_sampler_br1_5_500epoch
类别 0 图像数: 64
Generating batch 2/10...


100%|██████████| 10/10 [00:03<00:00,  2.67it/s, step=1, sample=1]


Generated 64 images of size torch.Size([3, 32, 32])
文件夹不存在，跳过类 0：data/cwru/test\0 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\0
文件夹不存在，跳过类 1：data/cwru/test\1 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\1
文件夹不存在，跳过类 2：data/cwru/test\2 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\2
文件夹不存在，跳过类 3：data/cwru/test\3 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\3
文件夹不存在，跳过类 4：data/cwru/test\4 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\4
文件夹不存在，跳过类 5：data/cwru/test\5 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\5
文件夹不存在，跳过类 6：data/cwru/test\6 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\6
文件夹不存在，跳过类 7：data/cwru/test\7 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\7
文件夹不存在，跳过类 8：data/cwru/test\8 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\8
文件夹不存在，跳过类 9：data/cwru/test\9 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\9
共删除 0 张生成图片。
检查图像数量：data/cwru_rf_result/cwru_sampler_br1_5_500epoch
类别 0 图像数: 64
类别 1 图像数: 64
Generating batch 3/10...


...

Generated 64 images of size torch.Size([3, 32, 32])
文件夹不存在，跳过类 0：data/cwru/test\0 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\0
文件夹不存在，跳过类 1：data/cwru/test\1 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\1
文件夹不存在，跳过类 2：data/cwru/test\2 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\2
文件夹不存在，跳过类 3：data/cwru/test\3 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\3
文件夹不存在，跳过类 4：data/cwru/test\4 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\4
文件夹不存在，跳过类 5：data/cwru/test\5 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\5
文件夹不存在，跳过类 6：data/cwru/test\6 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\6
文件夹不存在，跳过类 7：data/cwru/test\7 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\7
文件夹不存在，跳过类 8：data/cwru/test\8 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\8
文件夹不存在，跳过类 9：data/cwru/test\9 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\9
共删除 0 张生成图片。
检查图像数量：data/cwru_rf_result/cwru_sampler_br1_5_500epoch
类别 0 图像数: 64
类别 1 图像数: 64
类别 2 图像数: 64
类别 3 图像数: 64
类别 4 图像数

100%|██████████| 10/10 [00:03<00:00,  2.67it/s, step=1, sample=1]


Generated 64 images of size torch.Size([3, 32, 32])
文件夹不存在，跳过类 0：data/cwru/test\0 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\0
文件夹不存在，跳过类 1：data/cwru/test\1 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\1
文件夹不存在，跳过类 2：data/cwru/test\2 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\2
文件夹不存在，跳过类 3：data/cwru/test\3 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\3
文件夹不存在，跳过类 4：data/cwru/test\4 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\4
文件夹不存在，跳过类 5：data/cwru/test\5 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\5
文件夹不存在，跳过类 6：data/cwru/test\6 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\6
文件夹不存在，跳过类 7：data/cwru/test\7 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\7
文件夹不存在，跳过类 8：data/cwru/test\8 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\8
文件夹不存在，跳过类 9：data/cwru/test\9 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\9
共删除 0 张生成图片。
检查图像数量：data/cwru_rf_result/cwru_sampler_br1_5_500epoch
类别 0 图像数: 64
类别 1 图像数: 64
类别 2 图像数: 64
类别 3 图像数: 64
类别 4 图像数

100%|██████████| 10/10 [00:03<00:00,  2.66it/s, step=1, sample=1]


Generated 64 images of size torch.Size([3, 32, 32])
文件夹不存在，跳过类 0：data/cwru/test\0 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\0
文件夹不存在，跳过类 1：data/cwru/test\1 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\1
文件夹不存在，跳过类 2：data/cwru/test\2 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\2
文件夹不存在，跳过类 3：data/cwru/test\3 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\3
文件夹不存在，跳过类 4：data/cwru/test\4 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\4
文件夹不存在，跳过类 5：data/cwru/test\5 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\5
文件夹不存在，跳过类 6：data/cwru/test\6 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\6
文件夹不存在，跳过类 7：data/cwru/test\7 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\7
文件夹不存在，跳过类 8：data/cwru/test\8 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\8
文件夹不存在，跳过类 9：data/cwru/test\9 或 data/cwru_rf_result/cwru_sampler_br1_5_500epoch\9
共删除 0 张生成图片。
检查图像数量：data/cwru_rf_result/cwru_sampler_br1_5_500epoch
类别 0 图像数: 64
类别 1 图像数: 64
类别 2 图像数: 64
类别 3 图像数: 64
类别 4 图像数